In [1]:
    import abc, fnmatch, io, dataclasses, IPython, jinja2, inspect, collections, pygments, mistune, inspect, importlib, types, base64, string, traitlets
    from mimetypes import guess_type; guess = lambda x: guess_type(x)[0]
    with __import__('importnb').Notebook():
        try: 
            from .utils import format_images, finalize_ipython
        except: 
            from utils import format_images, finalize_ipython
    ip = IPython.get_ipython()

In [2]:
def strip_graphviz_headers(Source):
    str = Source._repr_svg_()
    # Mistune does a bad job of handling inline comments.
    for i in range(4): str = str.partition('>')[2]

    # Mistune has a hard time with the xmlns information.
    for year in (1999, 2000): str = str.replace(F'xmlns:xlink="http://www.w3.org/{year}/xlink"', '')
    return str.lstrip()

In [3]:
def graphviz(ip, str): 
    try: import graphviz
    except: return
    if '->' in str and not str.startswith('di'): str = 'di' + str
    return IPython.display.HTML(strip_graphviz_headers(graphviz.Source(str)))

In [4]:
def eval_shorthand_ipython(ip, str):
    return eval(ip.input_transformer_manager.transform_cell(str), ip.user_ns, ip.user_global_ns)

In [5]:
def flatten(str): return ''.join(str.splitlines())

In [6]:
def embed(ip, str):
    type = guess(str) or ''
    if str.startswith('http'): 
        if type.startswith('image') and not type.endswith('svg'):
            return F"""<img src="{str}"/>"""
        return F"""<iframe src="{str}" width="100%" height="500px"></iframe>"""

In [7]:
class StringFormatter(IPython.core.formatters.MimeBundleFormatter):
    mimebundle_formatter = traitlets.Instance(IPython.core.formatters.MimeBundleFormatter)
    formatters = traitlets.Dict()
    environment = traitlets.Instance(jinja2.Environment, kw={
        'finalize': finalize_ipython,
        'lstrip_blocks': True
    })
    level = traitlets.Int(default_value=0)
    @traitlets.default('formatters')
    def _default_formatters(self):return {
        'https://*' : embed,
        'http://*' : embed,
        'graph {*}': graphviz, 
        'digraph {*}': graphviz,
        ';[! ]*': eval_shorthand_ipython,
        ',[! ]*': eval_shorthand_ipython,
    }
    
    def __enter__(self): self.level += 1
    def __exit__(self, *exc): self.level -= 1
    
    def __call__(self, object, include=None, exclude=None, display=False):
        if isinstance(object, str):
            with self:
                for type, callable in self.formatters.items():
                    if fnmatch.fnmatch(object, type):
                        result = callable(self.parent, object)
                        if result: 
                            return ip.display_formatter.format(result)
                if self.level == 1: return self.format_html(object)
        return self.mimebundle_formatter(object, include, exclude)
    
    def format_html(self, object, **dict):
        import builtins
        self.environment.globals.update(vars(builtins))
        return {
            'text/html': self.environment.from_string(
                mistune.markdown(object, renderer=HighlightRenderer(), escape=False, parse_block_html=True, parse_inline_html=True)
            ).render(**(dict or self.parent.user_ns)), 'text/markdown': object, 'text/plain': object
        }, {}

IPython.core.formatters.FormatterABC.register(StringFormatter);

In [8]:
    lexers = set()

In [9]:
class HighlightRenderer(mistune.Renderer):
    def block_code(self, code, lang):
        global lexers
        lang = (lang or 'ipython').lstrip('%%')
        lexer = pygments.lexers.get_lexer_by_name(lang, stripall=True)
        formatter = pygments.formatters.html.HtmlFormatter()
        formatted = pygments.highlight(code, lexer, formatter)
        if 'lang' not in lexers:
            formatted += f"""<style>{formatter.get_style_defs('.highlight')}</style>"""
        return formatted

In [10]:
def format_doc(object, **dict):
    if object.__doc__:
        if inspect.isfunction(object) or inspect.isclass(object):
            dict.update(vars(importlib.import_module(object.__module__)))
        if inspect.ismodule(object):
            dict.update(vars(importlib.import_module(object.__name__)))
        return ip.display_formatter.mimebundle_formatter.format_html(object.__doc__, **dict)

In [11]:
   def show_axes(object):
        ip = IPython.get_ipython()
        import matplotlib.backends.backend_agg
        bytes = io.BytesIO()
        matplotlib.backends.backend_agg.FigureCanvasAgg(object.figure).print_png(bytes)
        bundle = {
            'image/png': bytes.getvalue()
        }
        return {
            'text/html': format_images('image/png', bundle),
            'text/plain': repr(object),
        }, {}

In [12]:
    def format_frame(DataFrame):
        return {
            'text/html': F"<span>{DataFrame._repr_html_().lstrip('<div>').rstrip('</div>')}</span>",
            'text/plain': repr(DataFrame)
        }, {}

In [13]:
def load_ipython_extension(ip): 
    ip = ip or IPython.get_ipython()
    ip.display_formatter.mimebundle_formatter.for_type_by_name('matplotlib.figure.Axes', show_axes)    
    ip.display_formatter.mimebundle_formatter.for_type_by_name('pandas.core.frame.DataFrame', format_frame)    
    ip.display_formatter.mimebundle_formatter.for_type(type, format_doc)
    ip.display_formatter.mimebundle_formatter.for_type(types.ModuleType, format_doc)

    ip.display_formatter.mimebundle_formatter = StringFormatter(
        mimebundle_formatter=ip.display_formatter.mimebundle_formatter, parent=ip)
    
def unload_ipython_extension(ip=None):
    ip.display_formatter.mimebundle_formatter = IPython.core.formatters.MimeBundleFormatter(parent=ip)
    

__name__ == '__main__' and load_ipython_extension(get_ipython())